In [1]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [5]:
import sklearn.linear_model as lm, pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean

In [12]:
LABEL = "like_count_ln"
INCLUDE_SVR = False

In [19]:
def get_data(url, drop=[]):
    import pandas as pd
    df = pd.read_csv(url)
    if len(drop) > 0:
        for col in drop:
            df.drop(columns=[col], inplace=True)
    return df

def bin_groups(df, percent=.05):
    import pandas as pd
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            for group, count in df[col].value_counts().iteritems():
                if count / len(df) < percent:
                    df.loc[df[col] == group, col] = 'Other'
    return df

def drop_columns_missing_data(df, cutoff=.5):
    import pandas as pd
    for col in df:
        if df[col].isna().sum() / len(df) > cutoff:
            df.drop(columns=[col], inplace=True)
    return df

def impute_mean(df):
    from sklearn.impute import SimpleImputer
    import pandas as pd, numpy as np
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
    return df

def impute_KNN(df):
    from sklearn.impute import KNNImputer
    from sklearn.preprocessing import MinMaxScaler
    import pandas as pd
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)
    df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns = df.columns)
    imp = KNNImputer(n_neighbors=5, weights="uniform")
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
    return df
        
def impute_reg(df):
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer
    import pandas as pd
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)
    imp = IterativeImputer(max_iter=10, random_state=12345)
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
    return df

def fs_variance(df, label="", p=0.8):
    from sklearn.feature_selection import VarianceThreshold
    import pandas as pd

    if label != "":
        X = df.drop(columns=[label])
    
    sel = VarianceThreshold(threshold=(p * (1 - p)))
    sel.fit(X)

    # Add the label back in after removing poor features
    return df[df.columns[sel.get_support(indices=True)]].join(df[label])

def fit_crossvalidate_mlr(df, k, label, repeat=True):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
    import pandas as pd
    from numpy import mean, std
    X = df.drop(label,axis=1)
    y = df[label]
    if repeat:
        cv = RepeatedKFold(n_splits=k, n_repeats=5, random_state=12345)
    else:
        cv = KFold(n_splits=k, random_state=12345, shuffle=True)
    scores = cross_val_score(LinearRegression(), X, y, scoring='r2', cv=cv, n_jobs=-1)
    print(f'Average R-squared:\t{mean(scores)}')
    return LinearRegression().fit(X, y)

def dump_pickle(model, file_name):
    import pickle
    pickle.dump(model, open(file_name, "wb"))

def load_pickle(file_name):
    import pickle
    model = pickle.load(open(file_name, "rb"))
    return model


In [20]:
# Data cleaning and preparation pipeline
df = get_data('noReplies-clean.csv')
df = bin_groups(df)
df = drop_columns_missing_data(df)
df = impute_mean(df)

# Feature selection and modeling pipeline
df = fs_variance(df, label=LABEL, p=.5)
model = fit_crossvalidate_mlr(df, 10, LABEL)

# Deployment pipeline
dump_pickle(model, 'saved_model.sav')

Average R-squared:	0.02536447581561319


In [22]:
fit = {}    # Use this to store each of the fit metrics
models = {} # Use this to store each of the models
        
# 1. LINEAR MODELS: assumes normal distribution, homoscedasticity, no multi-collinearity, independence, and no auto-correlation (some exceptions apply; some of these algorithms are better at handling violations of these assumptions)
import sklearn.linear_model as lm, pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean

# We will automate this later, just create
X = df.drop(columns=[LABEL])
y = df[LABEL]

# Set up a standard cross_validation object to use for each algorithm
cv = KFold(n_splits=5, random_state=12345, shuffle=True)

# 1.1. Ordinary Least Squares Multiple Linear Regression
model_ols = lm.LinearRegression()
fit['OLS'] = mean(cross_val_score(model_ols, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['OLS'] = model_ols

# 1.2. Ridge Regression: more robust to multi-collinearity
model_rr = lm.Ridge(alpha=0.5) # adjust this alpha parameter for better results (between 0 and 1)
fit['Ridge'] = mean(cross_val_score(model_rr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Ridge'] = model_rr

# 1.3. Lasso Regression: better for sparse values like RetweetCount where most are zeros but a few have many retweets.
model_lr = lm.Lasso(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
fit['Lasso'] = mean(cross_val_score(model_lr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Lasso'] = model_lr

# 1.4. Least Angle Regression: good when the number of features is greater than the number of samples
model_llr = lm.LassoLars(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
fit['LARS'] = mean(cross_val_score(model_llr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['LARS'] = model_llr

# 1.5. Bayesian Regression: probability based; allows regularization parameters, automatically tuned to data
model_br = lm.BayesianRidge()
fit['Bayesian'] = mean(cross_val_score(model_br, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Bayesian'] = model_br

# 1.6. Generalized Linear Regression (Poisson): Good for non-normal distribution, count-based data, and a Poisson distribution
model_pr = lm.TweedieRegressor(power=1, link="log") # Power=1 means this is a Poisson
fit['Poisson'] = mean(cross_val_score(model_pr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Poisson'] = model_pr

# 1.7. Generalized Linear Regression (Gamma): Good for non-normal distribution, continuous data, and a Gamma distribution
model_gr = lm.TweedieRegressor(power=2, link="log") # Power=2 means this is a Gamma
fit['Gamma'] = mean(cross_val_score(model_gr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Gamma'] = model_gr

# 1.8. Generalized Linear Regression (Inverse Gamma): Good non-normal distribution, continuous data, and an inverse Gamma distribution
model_igr = lm.TweedieRegressor(power=3) # Power=3 means this is an inverse Gamma
fit['Inverse'] = mean(cross_val_score(model_igr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Inverse'] = model_igr


# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

,R-squared
OLS,0.025416
Ridge,0.025416
Bayesian,0.025416
Lasso,0.025416
LARS,-0.000029
Poisson,-0.000029
Gamma,NaN
Inverse,NaN


In [ ]:
# SUPPORT VECTOR MACHINES: Ideal for noisy data with large gaps among values
from sklearn import svm

# 1.9. SVM: this is the default SVM, parameters can be modified to make this more accurate
model_svm = svm.SVR()
fit['SupportVM'] = mean(cross_val_score(model_svm, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['SupportVM'] = model_svm

In [26]:
# 1.10. Linear SVM: Faster than SVM but only considers a linear model
model_lsvm = svm.LinearSVR()
fit['Linear SVM'] = mean(cross_val_score(model_lsvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Linear SVM'] = model_lsvm

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
# 1.11. NuSVM: 
model_nusvm = svm.NuSVR()
fit['NuSupportVM'] = mean(cross_val_score(model_nusvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['NuSupportVM'] = model_nusvm

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

In [ ]:
# KNN: NEAREST NEIGHBORS REGRESSION
from sklearn import neighbors

# 1.12. KNeighborsRegressor: 
model_knnr = neighbors.KNeighborsRegressor(n_neighbors=10, weights='uniform')
fit['KNNeighbors'] = mean(cross_val_score(model_knnr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['KNNeighbors'] = model_knnr

# 1.13. KNeighborsRegressor: 
model_knnrd = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
fit['KNNeighborsD'] = mean(cross_val_score(model_knnrd, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['KNNeighborsD'] = model_knnrd

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)


In [4]:
# GAUSSIAN PROCESS REGRESSION
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

# 1.14. GaussianProcessRegressor:
model_gpr = gaussian_process.GaussianProcessRegressor(DotProduct() + WhiteKernel())
fit['GaussianP'] = mean(cross_val_score(model_gpr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['GaussianP'] = model_gpr

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

NameError: name 'mean' is not defined

In [ ]:
# DECISION TREE MODELS: no assumptions about the data
import sklearn.tree as tree
import sklearn.ensemble as se

# 1.15. Decision Tree Regression
model_dt = tree.DecisionTreeRegressor(random_state=12345)
fit['Dec Tree'] = mean(cross_val_score(model_dt, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Dec Tree'] = model_dt


# DECISION TREE-BASED ENSEMBLE MODELS: great for minimizing overfitting, these are based on averaging many unique sub-samples and combining algorithms 
# 1.16. Decision Forrest
model_df = se.RandomForestRegressor(random_state=12345)
fit['Dec Forest'] = mean(cross_val_score(model_df, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Dec Forest'] = model_df

# 1.17. ExtraTreesRegressor
model_etr = se.ExtraTreesRegressor(random_state=12345)
fit['Extra Trees'] = mean(cross_val_score(model_etr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Extra Trees'] = model_etr

# 1.18. AdaBoostRegressor
model_abr = se.AdaBoostRegressor(n_estimators=100, random_state=12345)
fit['AdaBoost DT'] = mean(cross_val_score(model_abr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['AdaBoost DT'] = model_abr

# 1.19. GradientBoostingRegressor
model_gbr = se.GradientBoostingRegressor(random_state=12345)
fit['Grad. Boost'] = mean(cross_val_score(model_gbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Grad. Boost'] = model_gbr

# 1.20. HistGradientBoostingRegressor
model_hgbr = se.HistGradientBoostingRegressor(random_state=12345)
fit['HG Boost'] = mean(cross_val_score(model_hgbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['HG Boost'] = model_hgbr

# 1.21. VotingRegressor: will combine other algorithms into an average; kind of cool
model_vr = se.VotingRegressor(estimators=[('DT', model_dt), ('DF', model_df), ('ETR', model_etr), ('ABR', model_abr), ('GBR', model_gbr)])
fit['Voting'] = mean(cross_val_score(model_vr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Voting'] = model_vr

# 1.22. StackingRegressor
from sklearn.linear_model import RidgeCV, LassoCV
estimators = [('ridge', RidgeCV()), ('lasso', LassoCV(random_state=42)), ('svr', svm.SVR(C=1, gamma=1e-6))]
model_sr = se.StackingRegressor(estimators=estimators, final_estimator=se.GradientBoostingRegressor(random_state=12345))
fit['Stacking'] = mean(cross_val_score(model_sr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Stacking'] = model_sr

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

In [ ]:
from xgboost import XGBRegressor

# 1.23. XGBRegressor
model_xgb = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['XGBoost'] = model_xgb

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)


In [ ]:
# NEURAL-NETWORK MODELS: Based on deep learning methods
import sklearn.neural_network as nn

# 1.23. MLPRegressor
model_nn = nn.MLPRegressor(max_iter=1000, random_state=12345) # Turn max_iter way up or down to get a more accurate result
fit['NeuralNet'] = mean(cross_val_score(model_nn, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['NeuralNet'] = model_nn

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)


In [ ]:
def fit_crossvalidate_reg(df, label, k=10, r=5, repeat=True):
    import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se
    from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
    from numpy import mean, std
    from sklearn import svm
    from sklearn import gaussian_process
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
    from xgboost import XGBRegressor

    X = df.drop(columns=[label])
    y = df[label]

    if repeat:
        cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
    else:
        cv = KFold(n_splits=k, random_state=12345, shuffle=True)
    
    fit = {}    # Use this to store each of the fit metrics
    models = {} # Use this to store each of the models

    # Create the model objects
    model_ols = lm.LinearRegression()
    model_rr = lm.Ridge(alpha=0.5) # adjust this alpha parameter for better results (between 0 and 1)
    model_lr = lm.Lasso(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
    model_llr = lm.LassoLars(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
    model_br = lm.BayesianRidge()
    model_pr = lm.TweedieRegressor(power=1, link="log") # Power=1 means this is a Poisson
    model_gr = lm.TweedieRegressor(power=2, link="log") # Power=2 means this is a Gamma
    model_igr = lm.TweedieRegressor(power=3) # Power=3 means this is an inverse Gamma
    model_svm = svm.SVR()
    model_lsvm = svm.LinearSVR()
    model_nusvm = svm.NuSVR()
    model_gpr = gaussian_process.GaussianProcessRegressor(DotProduct() + WhiteKernel())
    model_df = se.RandomForestRegressor(random_state=12345)
    model_etr = se.ExtraTreesRegressor(random_state=12345)
    model_abr = se.AdaBoostRegressor(n_estimators=100, random_state=12345)
    model_gbr = se.GradientBoostingRegressor(random_state=12345)
    model_hgbr = se.HistGradientBoostingRegressor(random_state=12345)
    model_vr = se.VotingRegressor(estimators=[('DF', model_df), ('ETR', model_etr), ('ABR', model_abr), ('GBR', model_gbr)])
    estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=42)), ('svr', svm.SVR(C=1, gamma=1e-6))]
    model_sr = se.StackingRegressor(estimators=estimators, final_estimator=se.GradientBoostingRegressor(random_state=12345))
    model_xgb = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
    
    # Fit a crss-validated R squared score and add it to the dict
    fit['OLS'] = mean(cross_val_score(model_ols, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Ridge'] = mean(cross_val_score(model_rr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Lasso'] = mean(cross_val_score(model_lr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['LARS'] = mean(cross_val_score(model_llr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Bayesian'] = mean(cross_val_score(model_br, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Poisson'] = mean(cross_val_score(model_pr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Gamma'] = mean(cross_val_score(model_gr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Inverse'] = mean(cross_val_score(model_igr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['SupportVM'] = mean(cross_val_score(model_svm, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Linear SVM'] = mean(cross_val_score(model_lsvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['NuSupportVM'] = mean(cross_val_score(model_nusvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['GaussianP'] = mean(cross_val_score(model_gpr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Dec Forest'] = mean(cross_val_score(model_df, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Extra Trees'] = mean(cross_val_score(model_etr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['AdaBoost DT'] = mean(cross_val_score(model_abr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Grad. Boost'] = mean(cross_val_score(model_gbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['HG Boost'] = mean(cross_val_score(model_hgbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Voting'] = mean(cross_val_score(model_vr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Stacking'] = mean(cross_val_score(model_sr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='r2', cv=cv, n_jobs=-1))

    # Add the model to another dict; make sure the keys have the same names as the list above
    models['OLS'] = model_ols
    models['Ridge'] = model_rr
    models['Lasso'] = model_lr
    models['LARS'] = model_llr
    models['Bayesian'] = model_br
    models['Poisson'] = model_pr
    models['Gamma'] = model_gr
    models['Inverse'] = model_igr
    models['SupportVM'] = model_svm
    models['Linear SVM'] = model_lsvm
    models['NuSupportVM'] = model_nusvm
    models['GaussianP'] = model_gpr
    models['Dec Forest'] = model_df
    models['Extra Trees'] = model_etr
    models['AdaBoost DT'] = model_abr
    models['Grad. Boost'] = model_gbr
    models['HG Boost'] = model_hgbr
    models['Voting'] = model_vr
    models['Stacking'] = model_sr
    models['XGBoost'] = model_xgb

    # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
    df_fit = pd.DataFrame({'R-squared':fit})
    df_fit.sort_values(by=['R-squared'], ascending=False, inplace=True)
    best_model = df_fit.index[0]
    print(df_fit)

    return models[best_model].fit(X, y)

# Now, see how this fits into the pipeline:---------------------------------------------
# Data cleaning and preparation pipeline
df = get_data('http://www.ishelp.info/data/housing_full.csv', ['Id'])
df = bin_groups(df)
df = drop_columns_missing_data(df)
df = impute_mean(df)

# Feature selection and modeling pipeline
df = fs_variance(df, label="SalePrice", p=.5)
model = fit_crossvalidate_reg(df, 'SalePrice', 5, 2)

# Deploy/store the model
dump_pickle(model, 'best_reg_model.sav')

In [ ]:
import pandas as pd
model = load_pickle('best_reg_model.sav')
print(pd.DataFrame({'Actual SalePrice':df.SalePrice, 'Predicted SalePrice':model.predict(df.drop(columns=['SalePrice']))}).head())